In [75]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

In [76]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join(r"C:\Users\indra\OneDrive\Documents\Indrani\UCB\w266\chatbot\data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")


b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [77]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

In [78]:
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [79]:
MAX_LENGTH = 10  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

In [80]:
datafile_validation = os.path.join(corpus, "formatted_movie_lines_valid.txt")

In [81]:
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

Start preparing training data ...
Reading lines...
Read 171282 sentence pairs
Trimmed to 49566 sentence pairs
Counting words...
Counted words: 15583


In [82]:
voc_validation, pairs_validation = loadPrepareData(corpus, corpus_name, datafile_validation, save_dir)

Start preparing training data ...
Reading lines...
Read 53424 sentence pairs
Trimmed to 15653 sentence pairs
Counting words...
Counted words: 7808


In [83]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


In [84]:
def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

In [85]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines

In [86]:
# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            utterance_id_pattern = re.compile('L[0-9]+')
            lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations

In [87]:
# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [88]:
# Define path to new file
datafile = r"C:\Users\indra\OneDrive\Documents\Indrani\UCB\w266\chatbot\data\cornell movie-dialogs corpus\formatted_movie_lines.txt"
datafile_validation = r"C:\Users\indra\OneDrive\Documents\Indrani\UCB\w266\chatbot\data\cornell movie-dialogs corpus\formatted_movie_lines_validation.txt"


In [89]:
delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

In [90]:
 # Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(r"C:\Users\indra\OneDrive\Documents\Indrani\UCB\w266\chatbot\data\cornell movie-dialogs corpus\movie_lines.txt", MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(r"C:\Users\indra\OneDrive\Documents\Indrani\UCB\w266\chatbot\data\cornell movie-dialogs corpus\movie_conversations.txt",
                              lines, MOVIE_CONVERSATIONS_FIELDS)

# conversations=conversations[0:50000]
# Write new csv file
number = 1
print("\nWriting Validation formatted file...")
with open(datafile_validation, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')

    for pair in extractSentencePairs(conversations):
        if number <= 50000:
            writer.writerow(pair)
        number += 1

number = 1
print("\nWriting Training  formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        if number > 50000:
            writer.writerow(pair)
        number += 1



Processing corpus...

Loading conversations...

Writing Validation formatted file...

Writing Training  formatted file...


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF ,LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [92]:
def save_model(dictionary, feature_names, filename):
    with open(filename, "wb") as f:
        np.savez(f, dictionary=dictionary, feature_names=feature_names)
        
def load_model(filename):
    """load model saved with save_model. Returns a tupe of dictionary, feature_names"""
    with open(filename, "rb") as f:
        arrs = np.load(f)
        return arrs["dictionary"], arrs["feature_names"]
    

def print_top_words(dictionary, feature_names, n_top_words):
    """print the top n_top_words from each topic in the dictionary matrix
    of dictionary. Feature names is a list of words with indices corresponding
    to the columns of the dictionary"""
    for topic_idx, topic in enumerate(dictionary):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


In [93]:
def train_model(data, n_features, n_components):
    model = Pipeline(steps=[("tfidf", TfidfVectorizer(max_df=0.95, min_df=2,
                                                      max_features=n_features, stop_words='english')),
                            ("lda", LatentDirichletAllocation(n_components=n_components, random_state=1))])
    model.fit(data)
    features = model["tfidf"].get_feature_names()
    dictionary = model["lda"].components_
    return dictionary, features

def train_save_model(data, n_features, n_components, filename):
    dictionary, features = train_model(data, n_features, n_components)
    save_model(dictionary, features, filename)
    return dictionary, features        

def load_news():
    dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                                 remove=('headers', 'footers', 'quotes'))
    data_samples = dataset.data
    return data_samples   


In [94]:
# number of top words to use in tfidf representation of data
n_features = 1000
# number of topics in LDA
n_components = 10

# train model and save it
print("Loading, training, and saving 20 news groups model")
news_data = load_news()
news_dict, news_feats = train_save_model(news_data, n_features, n_components, "data/topic/news-lda.npz")
print_top_words(news_dict, news_feats, 10)

Loading, training, and saving 20 news groups model
Topic #0: 00 10 11 1993 15 12 23 13 apr 17
Topic #1: god people think jesus don say believe just does know
Topic #2: windows window file program use files using edu server motif
Topic #3: government people key israel gun law don state encryption public
Topic #4: drive card mac software video monitor use windows pc scsi
Topic #5: space nasa new orbit earth air years launch research water
Topic #6: game team year games season players play hockey league win
Topic #7: mail phone address thanks sale edu email interested 00 price
Topic #8: just like car don bike good know think ve time
Topic #9: thanks know mail looking post hi info faq help appreciated



In [95]:
USE_CUDA = torch.cuda.is_available()
print("USE_CUDA", USE_CUDA)
if USE_CUDA:
    device = torch.device("cuda")

USE_CUDA True


In [96]:
def calculate_codes(topic_for_code, input_seq_for_code, voc, feature_path, batch_size):
    # batch_size = 64 for training, 1 for chatting
    ldadict = np.load(feature_path)["feature_names"]
    new_input_seq = torch.zeros(batch_size, len(ldadict))

    for i in range(batch_size):
        for j in range(len(input_seq_for_code[i])):
            input_seq_words = voc.index2word[input_seq_for_code[i][j].item()]
            for check_index in range(len(ldadict)):
                if ldadict[check_index] == input_seq_words:
                    new_input_seq[i][check_index] = 1

    three_d_topic = topic_for_code.repeat(batch_size, 1, 1).to(device)
    three_d_q = new_input_seq.repeat(1, 1, 1).permute(1, 2, 0).to(device)

    return torch.bmm(three_d_topic, three_d_q)


In [97]:
class EncoderRNN(nn.Module):
    def __init__(self,
                 hidden_size,
                 embedding,
                 topics,
                 n_layers=1,
                 dropout=0,
                 batch_size=64):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.topics = topics
        self.batch_size = batch_size
        # self.voc = voc
        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        self.gru = nn.GRU(hidden_size,
                          hidden_size,
                          n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        #print(input_seq)
        batch_size = 1
        #get the codes first
        input_seq_for_code = input_seq.transpose(0,1)
        topic_for_code = self.topics
        feature_path = r"data/topic/news-lda.npz"
        codes = calculate_codes(topic_for_code, input_seq_for_code, voc, feature_path, self.batch_size)
        #print(codes.size())
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden, codes


In [98]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)


In [99]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [100]:
def validation(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):
    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden, codes = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    teacher_forcing_ratio = 1.0
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs, codes, batch_size
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs, codes, batch_size
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    return sum(print_losses) / n_totals


In [101]:
def train(input_variable,
          lengths,
          target_variable,
          mask,
          max_target_len,
          encoder,
          decoder,
          embedding,
          encoder_optimizer,
          decoder_optimizer,
          batch_size,
          clip,
          max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden, codes = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    #decoder_hidden = encoder_hidden[:decoder.n_layers]
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    teacher_forcing_ratio = 1.0
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):

            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs, codes, batch_size
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs, codes, batch_size
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals


In [102]:
def trainIters(model_name,
               voc,
               voc_validation,
               pairs,
               pairs_validation,
               encoder,
               decoder,
               encoder_optimizer,
               decoder_optimizer,
               embedding,
               encoder_n_layers,
               decoder_n_layers,
               save_dir,
               n_iteration,
               batch_size,
               print_every,
               save_every,
               clip,
               corpus_name,
               loadFilename,
               DICT_NAME,
               checkpoint):

    #history record file
    history_file = open(r'history_delta.txt', 'w')

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    training_batches_validation = [batch2TrainData(voc_validation, [random.choice(pairs_validation) for _ in range(batch_size)])
                        for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    print_loss_validation = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        training_batch_validation = training_batches_validation[iteration - 1]

        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch
        input_variable_validation, lengths_validation, target_variable_validation, mask_validation, max_target_len_validation = training_batch_validation

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss
        loss_validation = validation(input_variable_validation, lengths_validation, target_variable_validation, mask_validation, max_target_len_validation, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss_validation += loss_validation

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print_loss_avg_validation = print_loss_validation / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Training loss: {:.4f}; Validation loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg, print_loss_avg_validation))
            history_file.write('{}'.format(print_loss_avg))
            history_file.write(' ')
            history_file.write('{}'.format(print_loss_avg_validation))
            history_file.write("\n")

            print_loss = 0
            print_loss_validation = 0



In [103]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            start_time = time.time()
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))
            end_time = time.time()
            print('Responding time:', end_time - start_time)
        except KeyError:
            print("Error: Encountered unknown word.")


In [104]:
def rand_pick(probabilities):
    x=random.uniform(0,1)
    #print(probabilities[0][20])
    cumulative_probability = 0.0
    #list = [n for n in range(len(probabilities))]
    for item in range(len(probabilities[0])):
        cumulative_probability += probabilities[0][item]
        if x < cumulative_probability:
            break
    item_list = torch.zeros([1], device=device, dtype=torch.long)
    item_list[0] = item
    score = torch.zeros([1],device=device)
    score[0]=probabilities[0][item]
    
    return item_list, score

class SearchDecoder(nn.Module):
    def __init__(self, encoder, decoder, batch_size=1):
        super(SearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.batch_size = batch_size

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden, codes = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self.decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input,
                                                          decoder_hidden,
                                                          encoder_outputs,
                                                          codes,
                                                          self.batch_size)
            # Obtain word based on probability distribution token and its softmax score
            decoder_input, decoder_scores = my_random_pick(decoder_output)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores


In [105]:
class TopicAttention(nn.Module):
    def __init__(self, topic_vocab_size, enc_hid_dim, dec_hid_dim):
        super(TopicAttention, self).__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn = nn.Linear(topic_vocab_size + dec_hid_dim + enc_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))

    def forward(self, hidden, topic_dict, enc_hidden):
        batch_size = enc_hidden.shape[0]
        num_topics = topic_dict.shape[0]

        hidden = hidden.repeat(num_topics, 1, 1).permute(1, 0, 2)
        enc_hidden = enc_hidden.repeat(num_topics, 1, 1).permute(1, 0, 2)
        topic_dict = topic_dict.repeat(batch_size, 1, 1)

        
        energy = torch.tanh(self.attn(torch.cat((hidden, topic_dict, enc_hidden), dim=2)))

        # energy = [batch_size, dec_hid_dim]
        energy = energy.permute(0, 2, 1)

        v = self.v.repeat(batch_size, 1).unsqueeze(1)

        # v = [batch_size, 1, dec_hid_dim]
        attention = torch.bmm(v, energy).squeeze(1)

        return F.softmax(attention, dim=1).unsqueeze(1)


In [106]:
class TopicDecoder(nn.Module):
    def __init__(self,
                 attn_model,
                 embedding,
                 hidden_size,
                 output_size,
                 enc_hid_dim,
                 dec_hid_dim,
                 topics,
                 topic_vocab_size,
                 n_layers=1,
                 dropout=0.1,
                 batch_size=1):
        super(TopicDecoder, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.topics = topics
        self.topic_vocab_size = topic_vocab_size
        self.batch_size = batch_size

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2 + topic_vocab_size + 1000, hidden_size) #1000 is len of topic dictionary
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)
        self.topic_attn = TopicAttention(topic_vocab_size, enc_hid_dim, dec_hid_dim)

    def forward(self,
                input_step,
                last_hidden,
                encoder_outputs,
                codes,
                batch_size=1):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)

        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        topic_attn_weights = self.topic_attn(last_hidden, self.topics, encoder_outputs[-1])

        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        encoder_outputs = encoder_outputs.permute(1,0,2)
      
        context = torch.bmm(attn_weights, encoder_outputs)
     
        topic_context = torch.bmm(topic_attn_weights, self.topics.repeat(batch_size, 1, 1))
       
        topic_for_Pk = self.topics.repeat(batch_size,1,1).permute(0,2,1)
        # print('topic_for_Pk',topic_for_Pk.size())
        Pk_context = torch.bmm(topic_for_Pk, codes)
        # print('Pk_context',Pk_context.size())

        # Concatenate weighted context vector and GRU output using Luong eq. 5
       
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        topic_context = topic_context.squeeze(1)
      
        Pk_context = Pk_context.squeeze(2)
        concat_input = torch.cat((rnn_output, context, topic_context, Pk_context), 1)
        # print('concat_input', concat_input.size())
        concat_output = torch.tanh(self.concat(concat_input))

        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [107]:
# Configure models
model_name = 'topic_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 1
decoder_n_layers = 1
dropout = 0.1
batch_size = 64 # 64 for training, 1 for chatting

In [108]:
DICT_NAME = 'data/topic/news'
DICT_PATH = DICT_NAME + '-lda.npz'
print("DICT_PATH",DICT_PATH)
topic_dict = torch.tensor(np.load(DICT_PATH)["dictionary"], dtype=torch.float).to(device)

DICT_PATH data/topic/news-lda.npz


In [109]:
# Set checkpoint to load from
loadFilename = None
checkpoint_iter = 64000

In [110]:
print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)

# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size=hidden_size,
                             embedding=embedding,
                             topics=topic_dict,
                             n_layers=encoder_n_layers,
                             dropout=dropout,
                             batch_size=batch_size)

enc_hid_dim, dec_hid_dim, emb_dim = hidden_size, hidden_size, hidden_size

decoder = TopicDecoder(attn_model,
                                  embedding,
                                  hidden_size,
                                  voc.num_words,
                                  enc_hid_dim,
                                  dec_hid_dim,
                                  topic_dict,
                                  topic_dict.shape[1],
                                  decoder_n_layers,
                                  dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)

# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [111]:
#start training
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 100

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name=model_name,
                   voc=voc,
                   voc_validation=voc_validation,
                   pairs=pairs,
                   pairs_validation=pairs_validation,
                   encoder=encoder,
                   decoder=decoder,
                   encoder_optimizer=encoder_optimizer,
                   decoder_optimizer=decoder_optimizer,
                   embedding=embedding,
                   encoder_n_layers=encoder_n_layers,
                   decoder_n_layers=decoder_n_layers,
                   save_dir=save_dir,
                   n_iteration=n_iteration,
                   batch_size=batch_size,
                   print_every=print_every,
                   save_every=save_every,
                   clip=clip,
                   corpus_name=corpus_name,
                   loadFilename=loadFilename,
                   DICT_NAME=DICT_NAME,
                   checkpoint=None)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Training loss: 9.6927; Validation loss: 8.8661
Iteration: 2; Percent complete: 0.1%; Training loss: 8.6254; Validation loss: 8.6060
Iteration: 3; Percent complete: 0.1%; Training loss: 8.3154; Validation loss: 8.4682
Iteration: 4; Percent complete: 0.1%; Training loss: 8.0159; Validation loss: 8.1137
Iteration: 5; Percent complete: 0.1%; Training loss: 7.8199; Validation loss: 8.1513
Iteration: 6; Percent complete: 0.1%; Training loss: 7.4927; Validation loss: 7.9734
Iteration: 7; Percent complete: 0.2%; Training loss: 7.3132; Validation loss: 7.7576
Iteration: 8; Percent complete: 0.2%; Training loss: 6.8961; Validation loss: 7.9036
Iteration: 9; Percent complete: 0.2%; Training loss: 6.9288; Validation loss: 7.8743
Iteration: 10; Percent complete: 0.2%; Training loss: 6.9273; Validation loss: 7.6272
Iteration: 11; Percent complete: 0.3%; Training loss: 6.6716; Validation loss

Iteration: 96; Percent complete: 2.4%; Training loss: 5.2090; Validation loss: 7.5012
Iteration: 97; Percent complete: 2.4%; Training loss: 5.1696; Validation loss: 7.4091
Iteration: 98; Percent complete: 2.5%; Training loss: 5.2261; Validation loss: 7.4273
Iteration: 99; Percent complete: 2.5%; Training loss: 5.3782; Validation loss: 7.4700
Iteration: 100; Percent complete: 2.5%; Training loss: 5.1777; Validation loss: 7.3874
Iteration: 101; Percent complete: 2.5%; Training loss: 5.2980; Validation loss: 7.3899
Iteration: 102; Percent complete: 2.5%; Training loss: 5.3402; Validation loss: 7.3254
Iteration: 103; Percent complete: 2.6%; Training loss: 5.2163; Validation loss: 7.7297
Iteration: 104; Percent complete: 2.6%; Training loss: 5.2175; Validation loss: 7.5023
Iteration: 105; Percent complete: 2.6%; Training loss: 5.3244; Validation loss: 7.5709
Iteration: 106; Percent complete: 2.6%; Training loss: 5.2211; Validation loss: 7.4021
Iteration: 107; Percent complete: 2.7%; Trainin

Iteration: 191; Percent complete: 4.8%; Training loss: 5.2445; Validation loss: 7.5980
Iteration: 192; Percent complete: 4.8%; Training loss: 5.2056; Validation loss: 7.6021
Iteration: 193; Percent complete: 4.8%; Training loss: 5.1456; Validation loss: 7.1169
Iteration: 194; Percent complete: 4.9%; Training loss: 5.2405; Validation loss: 7.4026
Iteration: 195; Percent complete: 4.9%; Training loss: 5.2292; Validation loss: 7.6006
Iteration: 196; Percent complete: 4.9%; Training loss: 5.0408; Validation loss: 7.4731
Iteration: 197; Percent complete: 4.9%; Training loss: 5.1386; Validation loss: 7.5594
Iteration: 198; Percent complete: 5.0%; Training loss: 5.3031; Validation loss: 7.6250
Iteration: 199; Percent complete: 5.0%; Training loss: 5.1498; Validation loss: 7.5452
Iteration: 200; Percent complete: 5.0%; Training loss: 5.1174; Validation loss: 7.1607
Iteration: 201; Percent complete: 5.0%; Training loss: 5.2292; Validation loss: 7.4229
Iteration: 202; Percent complete: 5.1%; Tra

Iteration: 286; Percent complete: 7.1%; Training loss: 5.0943; Validation loss: 7.5156
Iteration: 287; Percent complete: 7.2%; Training loss: 5.3101; Validation loss: 7.3877
Iteration: 288; Percent complete: 7.2%; Training loss: 5.4109; Validation loss: 7.5594
Iteration: 289; Percent complete: 7.2%; Training loss: 5.3191; Validation loss: 7.4714
Iteration: 290; Percent complete: 7.2%; Training loss: 5.2106; Validation loss: 7.4162
Iteration: 291; Percent complete: 7.3%; Training loss: 5.0473; Validation loss: 7.3196
Iteration: 292; Percent complete: 7.3%; Training loss: 5.1000; Validation loss: 7.5980
Iteration: 293; Percent complete: 7.3%; Training loss: 5.2228; Validation loss: 7.4368
Iteration: 294; Percent complete: 7.3%; Training loss: 5.1325; Validation loss: 7.3275
Iteration: 295; Percent complete: 7.4%; Training loss: 5.1854; Validation loss: 7.6276
Iteration: 296; Percent complete: 7.4%; Training loss: 5.2726; Validation loss: 7.4364
Iteration: 297; Percent complete: 7.4%; Tra

Iteration: 381; Percent complete: 9.5%; Training loss: 4.9532; Validation loss: 7.5244
Iteration: 382; Percent complete: 9.6%; Training loss: 5.3752; Validation loss: 7.2984
Iteration: 383; Percent complete: 9.6%; Training loss: 5.4404; Validation loss: 7.5133
Iteration: 384; Percent complete: 9.6%; Training loss: 4.9760; Validation loss: 7.3942
Iteration: 385; Percent complete: 9.6%; Training loss: 5.0143; Validation loss: 7.5660
Iteration: 386; Percent complete: 9.7%; Training loss: 5.1966; Validation loss: 7.3180
Iteration: 387; Percent complete: 9.7%; Training loss: 5.2436; Validation loss: 7.4537
Iteration: 388; Percent complete: 9.7%; Training loss: 5.0836; Validation loss: 7.4817
Iteration: 389; Percent complete: 9.7%; Training loss: 5.0483; Validation loss: 7.6227
Iteration: 390; Percent complete: 9.8%; Training loss: 5.3567; Validation loss: 7.3444
Iteration: 391; Percent complete: 9.8%; Training loss: 5.2641; Validation loss: 7.2347
Iteration: 392; Percent complete: 9.8%; Tra

...

Iteration: 3549; Percent complete: 88.7%; Training loss: 5.2096; Validation loss: 7.8644
Iteration: 3550; Percent complete: 88.8%; Training loss: 5.1058; Validation loss: 7.4205
Iteration: 3551; Percent complete: 88.8%; Training loss: 5.2444; Validation loss: 7.1448
Iteration: 3552; Percent complete: 88.8%; Training loss: 5.1765; Validation loss: 7.5085
Iteration: 3553; Percent complete: 88.8%; Training loss: 5.0967; Validation loss: 7.0904
Iteration: 3554; Percent complete: 88.8%; Training loss: 5.1850; Validation loss: 7.4009
Iteration: 3555; Percent complete: 88.9%; Training loss: 5.2112; Validation loss: 7.3762
Iteration: 3556; Percent complete: 88.9%; Training loss: 5.1887; Validation loss: 7.6034
Iteration: 3557; Percent complete: 88.9%; Training loss: 5.2738; Validation loss: 7.6576
Iteration: 3558; Percent complete: 88.9%; Training loss: 4.8145; Validation loss: 7.4660
Iteration: 3559; Percent complete: 89.0%; Training loss: 5.1785; Validation loss: 7.5882
Iteration: 3560; Perc

Iteration: 3642; Percent complete: 91.0%; Training loss: 5.1144; Validation loss: 7.4131
Iteration: 3643; Percent complete: 91.1%; Training loss: 5.0127; Validation loss: 7.2218
Iteration: 3644; Percent complete: 91.1%; Training loss: 5.0460; Validation loss: 7.9476
Iteration: 3645; Percent complete: 91.1%; Training loss: 4.8943; Validation loss: 7.4086
Iteration: 3646; Percent complete: 91.1%; Training loss: 4.9477; Validation loss: 7.5701
Iteration: 3647; Percent complete: 91.2%; Training loss: 4.9931; Validation loss: 7.7314
Iteration: 3648; Percent complete: 91.2%; Training loss: 5.0788; Validation loss: 7.4791
Iteration: 3649; Percent complete: 91.2%; Training loss: 5.0628; Validation loss: 7.1873
Iteration: 3650; Percent complete: 91.2%; Training loss: 5.2739; Validation loss: 7.3115
Iteration: 3651; Percent complete: 91.3%; Training loss: 5.1187; Validation loss: 7.3631
Iteration: 3652; Percent complete: 91.3%; Training loss: 5.0280; Validation loss: 7.3694
Iteration: 3653; Perc

Iteration: 3735; Percent complete: 93.4%; Training loss: 5.1141; Validation loss: 7.8173
Iteration: 3736; Percent complete: 93.4%; Training loss: 5.0272; Validation loss: 7.2566
Iteration: 3737; Percent complete: 93.4%; Training loss: 5.2130; Validation loss: 7.5119
Iteration: 3738; Percent complete: 93.5%; Training loss: 5.1715; Validation loss: 7.4300
Iteration: 3739; Percent complete: 93.5%; Training loss: 5.0222; Validation loss: 7.9566
Iteration: 3740; Percent complete: 93.5%; Training loss: 4.9697; Validation loss: 7.5417
Iteration: 3741; Percent complete: 93.5%; Training loss: 5.0893; Validation loss: 7.2328
Iteration: 3742; Percent complete: 93.5%; Training loss: 5.0827; Validation loss: 7.2793
Iteration: 3743; Percent complete: 93.6%; Training loss: 5.2738; Validation loss: 7.3617
Iteration: 3744; Percent complete: 93.6%; Training loss: 5.2547; Validation loss: 7.1987
Iteration: 3745; Percent complete: 93.6%; Training loss: 4.8189; Validation loss: 7.4360
Iteration: 3746; Perc

Iteration: 3828; Percent complete: 95.7%; Training loss: 5.2583; Validation loss: 7.5036
Iteration: 3829; Percent complete: 95.7%; Training loss: 5.0310; Validation loss: 7.4193
Iteration: 3830; Percent complete: 95.8%; Training loss: 5.1333; Validation loss: 7.6142
Iteration: 3831; Percent complete: 95.8%; Training loss: 5.1254; Validation loss: 7.7445
Iteration: 3832; Percent complete: 95.8%; Training loss: 5.2173; Validation loss: 7.4572
Iteration: 3833; Percent complete: 95.8%; Training loss: 5.2195; Validation loss: 7.5020
Iteration: 3834; Percent complete: 95.9%; Training loss: 5.3115; Validation loss: 7.3750
Iteration: 3835; Percent complete: 95.9%; Training loss: 4.9079; Validation loss: 7.5583
Iteration: 3836; Percent complete: 95.9%; Training loss: 5.1973; Validation loss: 7.6615
Iteration: 3837; Percent complete: 95.9%; Training loss: 5.0961; Validation loss: 7.2981
Iteration: 3838; Percent complete: 96.0%; Training loss: 5.0405; Validation loss: 7.3874
Iteration: 3839; Perc

Iteration: 3921; Percent complete: 98.0%; Training loss: 5.2929; Validation loss: 7.7666
Iteration: 3922; Percent complete: 98.0%; Training loss: 5.1495; Validation loss: 7.7524
Iteration: 3923; Percent complete: 98.1%; Training loss: 5.1337; Validation loss: 7.2241
Iteration: 3924; Percent complete: 98.1%; Training loss: 5.1384; Validation loss: 6.9185
Iteration: 3925; Percent complete: 98.1%; Training loss: 4.9437; Validation loss: 7.5617
Iteration: 3926; Percent complete: 98.2%; Training loss: 4.9831; Validation loss: 7.5120
Iteration: 3927; Percent complete: 98.2%; Training loss: 4.9698; Validation loss: 7.9640
Iteration: 3928; Percent complete: 98.2%; Training loss: 5.3163; Validation loss: 7.3512
Iteration: 3929; Percent complete: 98.2%; Training loss: 5.0368; Validation loss: 7.1735
Iteration: 3930; Percent complete: 98.2%; Training loss: 5.0202; Validation loss: 7.3679
Iteration: 3931; Percent complete: 98.3%; Training loss: 4.9041; Validation loss: 7.4794
Iteration: 3932; Perc

In [112]:
PATH=r"C:\\Users\\indra\\OneDrive\\Documents\\Indrani\\UCB\\w266\\chatbot"
print(PATH)
torch.save(model_name, PATH)

C:\\Users\\indra\\OneDrive\\Documents\\Indrani\\UCB\\w266\\chatbot


PermissionError: [Errno 13] Permission denied: 'C:\\\\Users\\\\indra\\\\OneDrive\\\\Documents\\\\Indrani\\\\UCB\\\\w266\\\\chatbot'

In [ ]:
import json
def save_current_config(save_model_file_name):
  config_dict = {'start_token':1, 'end_token':2, 'save_model_file_name':save_model_file_name}
  config_filename = PATH + save_model_file_name + '\config.txt'
  with open(config_filename, 'w') as f:
    json.dump(config_dict, f)

In [ ]:
save_current_config("")

In [ ]:
import time

In [ ]:
encoder.eval()
decoder.eval()
searcher = SearchDecoder(encoder, decoder)
# evaluateInput(encoder, decoder, searcher, voc)